In [1]:
import os
from sys import maxsize
import numpy as np
import matplotlib.pyplot as plt

100 per file * 10 files

In [68]:
def readdat(directory):
    Us = []
    Vs = []
    As = []
    
    for filenum in range(10):
        # print(directory + filename_pre + str(filenum))

        with open(directory + filename_pre + str(filenum), 'r') as file:
            for c in range(100):
                line = file.readline().split()
                # print(line)
                u_size = (int(line[0]), int(line[1]))
                utemp = []
                for i in range(u_size[0]):
                    utemp.append([float(x) for x in file.readline().split()])
                Us.append(utemp)
                # print(utemp)

                line = file.readline().split()
                # print(line)
                v_size = (int(line[0]), int(line[1]))
                vtemp = []
                for i in range(v_size[0]):
                    vtemp.append([float(x) for x in file.readline().split()])
                Vs.append(vtemp)
                # print(vtemp)

                line = file.readline().split()
                # print(line)
                alloc_size = (int(line[0]), int(line[1]))
                alloctemp = []
                for i in range(alloc_size[0]):
                    alloctemp.append([int(x) for x in file.readline().split()])
                As.append(alloctemp)
                # print(alloctemp)

                line = file.readline()
                # print(line)
    return (Us, Vs, As)

EF calculations

In [3]:
additive_utililty = lambda alloc, utility : sum(np.array(alloc) * np.array(utility))

def complementing_utility(alloc, utility, p_utility) :
    pairwise_alloc = []
    for i in range(0, len(alloc), 2):
        pairwise_alloc.append(alloc[i] & alloc[i+1])
    return additive_utililty(alloc, utility) + additive_utililty(pairwise_alloc, p_utility)

def envyMap(allocs, utilities, p_utilities):
    size_agents = np.array(allocs).shape[0]
    envy_map = np.zeros((size_agents, size_agents))
    for i in range(len(envy_map)):
        for j in range(len(envy_map[i])):
            envy_map[i][j] = complementing_utility(allocs[i], utilities[i], p_utilities[i]) \
            - complementing_utility(allocs[j], utilities[i], p_utilities[i])
    return envy_map

def envyMap_upto1(allocs, utilities, p_utilities):
    size_agents = np.array(allocs).shape[0]
    size_res = np.array(allocs).shape[1]
    envy_map = np.zeros((size_agents, size_agents))
    for i in range(len(envy_map)) :
        for j in range(len(envy_map[i])):
            envy_map[i][j] = complementing_utility(allocs[i], utilities[i], p_utilities[i]) \
            - complementing_utility(allocs[j], utilities[i], p_utilities[i])
            if envy_map[i][j] < 0 :
                envy_res_drop1_j = np.zeros(size_res)
                max_pos_envy = - maxsize
                for x in range(len(envy_res_drop1_j)) :
                    new_alloc_j = list(allocs[j])
                    new_alloc_j[x] = 0
                    envy_res_drop1_j[x] = complementing_utility(allocs[i], utilities[i], p_utilities[i]) \
                    - complementing_utility(new_alloc_j, utilities[i], p_utilities[i])
                    # print(envy_res_drop1_j)
                    if envy_res_drop1_j[x] >= max_pos_envy:
                        envy_map[i][j] = envy_res_drop1_j[x]
                        max_pos_envy = envy_res_drop1_j[x]
                
    return envy_map

def envyMap_upto2(allocs, utilities, p_utilities):
    size_agents = np.array(allocs).shape[0]
    size_res = np.array(allocs).shape[1]
    envy_map = np.zeros((size_agents, size_agents))
    for i in range(len(envy_map)) :
        for j in range(len(envy_map[i])):
            envy_map[i][j] = complementing_utility(allocs[i], utilities[i], p_utilities[i]) \
            - complementing_utility(allocs[j], utilities[i], p_utilities[i])
            if envy_map[i][j] < 0 :
                envy_res_drop1_j = np.zeros(size_res)
                max_pos_envy = - maxsize
                for x in range(len(envy_res_drop1_j)) :
                    new_alloc_j = list(allocs[j])
                    new_alloc_j[x] = 0
                    envy_res_drop1_j[x] = complementing_utility(allocs[i], utilities[i], p_utilities[i]) \
                    - complementing_utility(new_alloc_j, utilities[i], p_utilities[i])
                    # print(envy_res_drop1_j)
                    if envy_res_drop1_j[x] >= max_pos_envy:
                        envy_map[i][j] = envy_res_drop1_j[x]
                        max_pos_envy = envy_res_drop1_j[x]
                        if envy_map[i][j] < 0 :
                            envy_res_drop2_j = np.zeros((size_res, size_res))
                            max_pos_envy_drop2 = - maxsize
                            for p in range(len(envy_res_drop2_j)):
                                for q in range(len(envy_res_drop2_j[p])):
                                    if p == q:
                                        continue
                                    new_alloc_j_drop2 = list(allocs[j])
                                    new_alloc_j_drop2[p] = 0
                                    new_alloc_j_drop2[q] = 0
                                    envy_res_drop2_j[p][q] = complementing_utility(allocs[i], utilities[i], p_utilities[i]) \
                                    - complementing_utility(new_alloc_j_drop2, utilities[i], p_utilities[i])
                                    if envy_res_drop2_j[p][q] >= max_pos_envy_drop2:
                                        envy_map[i][j] = envy_res_drop2_j[p][q]
                                        max_pos_envy_drop2 = envy_res_drop2_j[p][q]
                
    return envy_map

In [81]:
def ef_percent(dir_name):
    Us, Vs, As = readdat(dir_name)
    Us = np.array(Us)
    Vs = np.array(Vs)
    As = np.array(As)
    print("Us shape :", Us.shape)
    print("Vs shape :", Vs.shape)
    print("As shape :", As.shape)
    
    ef_map = []
    envymap0 = []
    for i in range(1000):
        envymaptemp = envyMap(As[i], Us[i], Vs[i])
        envymap0.append(envymaptemp)
        # print(envymaptemp)
        ef_map.append(not bool(np.sum(envymaptemp<0)))

    ef_map = np.array(ef_map)
    envymap0 = np.array(envymap0)
    # print(ef_map)
    print("EF0 percent : ", np.sum(ef_map)/len(ef_map) * 100)

    ef1_map = []
    envymap1 = []
    for i in range(1000):
        envymaptemp = envyMap_upto1(As[i], Us[i], Vs[i])
        envymap1.append(envymaptemp)
        # print(envymaptemp)
        ef1_map.append(not bool(np.sum(envymaptemp<0)))

    ef1_map = np.array(ef1_map)
    envymap1 = np.array(envymap1)
    # print(ef_map)
    print("EF1 percent : ", np.sum(ef1_map)/len(ef1_map) * 100)

    ef2_map = []
    envymap2 = []
    for i in range(1000):
        envymaptemp = envyMap_upto2(As[i], Us[i], Vs[i])
        envymap2.append(envymaptemp)
        # print(envymaptemp)
        ef2_map.append(not bool(np.sum(envymaptemp<0)))

    ef2_map = np.array(ef2_map)
    envymap2 = np.array(envymap2)
    # print(ef_map)
    print("EF2 percent : ", np.sum(ef2_map)/len(ef2_map) * 100)
    
    print("Means: envy")
    print(np.mean(envymap0, axis=0))
    print(np.mean(envymap1, axis=0))
    print(np.mean(envymap2, axis=0))
    
    print("Means: max(0, envy)")
    print(np.mean(envymap0.clip(min=0), axis=0))
    print(np.mean(envymap1.clip(min=0), axis=0))
    print(np.mean(envymap2.clip(min=0), axis=0))
        
    print("Means: min(0, envy)")
    print(np.mean(envymap0.clip(max=0), axis=0))
    print(np.mean(envymap1.clip(max=0), axis=0))
    print(np.mean(envymap2.clip(max=0), axis=0))
    
    print("Stds")
    print(np.std(envymap0, axis=0))
    print(np.std(envymap1, axis=0))
    print(np.std(envymap2, axis=0))
    
    return (envymap0, envymap1, envymap2)


For additive utilities

In [82]:
dir_u_r10_u2 = "/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_u_r10_u2/"
dir_uv_r10_u2 = "/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_uv_r10_u2/"

dir_u_r4_u2 = "/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_u_r4_u2/"
dir_uv_r4_u2 = "/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_uv_r4_u2/"

dir_u_r6_u2 = "/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_u_r6_u2/"
dir_uv_r6_u2 = "/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_uv_r6_u2/"

dir_uv_r6_u4 = "/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_uv_r6_u4/"

dir_u_r8_u2 = "/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_u_r8_u2/"
dir_uv_r8_u2 = "/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_uv_r8_u2/"

filename_pre = "nash_allocs_random_uv_1"

For complementing utilities

In [8]:
# failtemp = np.argwhere(ef1_map==False)[0][0]
# print(failtemp)
# print(Us[failtemp])
# print(Vs[failtemp])
# print(As[failtemp])
# print(envymap1[failtemp])

In [84]:
(en0, en1, en2) = ef_percent(dir_u_r10_u2)

Us shape : (1000, 2, 10)
Vs shape : (1000, 2, 5)
As shape : (1000, 2, 10)
EF0 percent :  99.9
EF1 percent :  100.0
EF2 percent :  100.0
Means: envy
[[  0.         156.83334623]
 [155.02036867   0.        ]]
[[  0.         156.83334623]
 [155.11827364   0.        ]]
[[  0.         156.83334623]
 [155.11827364   0.        ]]
Means: max(0, envy)
[[  0.         156.83334623]
 [155.02114089   0.        ]]
[[  0.         156.83334623]
 [155.11827364   0.        ]]
[[  0.         156.83334623]
 [155.11827364   0.        ]]
Means: min(0, envy)
[[ 0.          0.        ]
 [-0.00077223  0.        ]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
Stds
[[ 0.         68.37707329]
 [65.3157873   0.        ]]
[[ 0.         68.37707329]
 [65.15536916  0.        ]]
[[ 0.         68.37707329]
 [65.15536916  0.        ]]


In [85]:
(en0, en1, en2) = ef_percent(dir_u_r8_u2)

Us shape : (1000, 2, 8)
Vs shape : (1000, 2, 4)
As shape : (1000, 2, 8)
EF0 percent :  98.1
EF1 percent :  100.0
EF2 percent :  100.0
Means: envy
[[  0.         126.33334216]
 [122.9806306    0.        ]]
[[  0.         127.21740552]
 [123.58211117   0.        ]]
[[  0.         127.21740552]
 [123.58211117   0.        ]]
Means: max(0, envy)
[[  0.         126.48942761]
 [123.09040063   0.        ]]
[[  0.         127.21740552]
 [123.58211117   0.        ]]
[[  0.         127.21740552]
 [123.58211117   0.        ]]
Means: min(0, envy)
[[ 0.         -0.15608545]
 [-0.10977003  0.        ]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
Stds
[[ 0.        64.1111858]
 [62.878994   0.       ]]
[[ 0.         62.71469491]
 [61.93159327  0.        ]]
[[ 0.         62.71469491]
 [61.93159327  0.        ]]


In [86]:
(en0, en1, en2) = ef_percent(dir_u_r6_u2)

Us shape : (1000, 2, 6)
Vs shape : (1000, 2, 3)
As shape : (1000, 2, 6)
EF0 percent :  93.5
EF1 percent :  100.0
EF2 percent :  100.0
Means: envy
[[ 0.         91.35345389]
 [88.96794779  0.        ]]
[[ 0.         94.27388561]
 [91.05224569  0.        ]]
[[ 0.         94.27388561]
 [91.05224569  0.        ]]
Means: max(0, envy)
[[ 0.         91.77398598]
 [89.44611016  0.        ]]
[[ 0.         94.27388561]
 [91.05224569  0.        ]]
[[ 0.         94.27388561]
 [91.05224569  0.        ]]
Means: min(0, envy)
[[ 0.         -0.42053209]
 [-0.47816237  0.        ]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
Stds
[[ 0.         55.49371266]
 [53.04071767  0.        ]]
[[ 0.         52.00652743]
 [50.2832711   0.        ]]
[[ 0.         52.00652743]
 [50.2832711   0.        ]]


In [87]:
(en0, en1, en2) = ef_percent(dir_u_r4_u2)

Us shape : (1000, 2, 4)
Vs shape : (1000, 2, 2)
As shape : (1000, 2, 4)
EF0 percent :  77.10000000000001
EF1 percent :  100.0
EF2 percent :  100.0
Means: envy
[[ 0.         56.20214644]
 [56.41082445  0.        ]]
[[ 0.         65.47889405]
 [64.33766935  0.        ]]
[[ 0.         65.47889405]
 [64.33766935  0.        ]]
Means: max(0, envy)
[[ 0.         58.45416904]
 [58.44216028  0.        ]]
[[ 0.         65.47889405]
 [64.33766935  0.        ]]
[[ 0.         65.47889405]
 [64.33766935  0.        ]]
Means: min(0, envy)
[[ 0.         -2.2520226 ]
 [-2.03133583  0.        ]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
Stds
[[ 0.         47.97520014]
 [45.98858705  0.        ]]
[[ 0.         39.40358954]
 [38.34352994  0.        ]]
[[ 0.         39.40358954]
 [38.34352994  0.        ]]


In [88]:
(en0, en1, en2) = ef_percent(dir_uv_r10_u2)

Us shape : (1000, 2, 10)
Vs shape : (1000, 2, 5)
As shape : (1000, 2, 10)
EF0 percent :  75.7
EF1 percent :  99.9
EF2 percent :  100.0
Means: envy
[[  0.         137.73304048]
 [134.82639453   0.        ]]
[[  0.         153.9664558 ]
 [152.27546644   0.        ]]
[[  0.         154.10125936]
 [152.27546644   0.        ]]
Means: max(0, envy)
[[  0.         142.92624834]
 [140.11198442   0.        ]]
[[  0.         153.99764655]
 [152.27546644   0.        ]]
[[  0.         154.10125936]
 [152.27546644   0.        ]]
Means: min(0, envy)
[[ 0.         -5.19320787]
 [-5.2855899   0.        ]]
[[ 0.         -0.03119075]
 [ 0.          0.        ]]
[[0. 0.]
 [0. 0.]]
Stds
[[  0.         113.72814897]
 [110.42648228   0.        ]]
[[ 0.         94.89523649]
 [90.41461064  0.        ]]
[[ 0.         94.72771526]
 [90.41461064  0.        ]]


In [89]:
(en0, en1, en2) = ef_percent(dir_uv_r8_u2)

Us shape : (1000, 2, 8)
Vs shape : (1000, 2, 4)
As shape : (1000, 2, 8)
EF0 percent :  77.8
EF1 percent :  100.0
EF2 percent :  100.0
Means: envy
[[  0.         104.18248343]
 [104.58306424   0.        ]]
[[  0.         119.98533569]
 [120.19430845   0.        ]]
[[  0.         119.98533569]
 [120.19430845   0.        ]]
Means: max(0, envy)
[[  0.         107.76165509]
 [108.48607588   0.        ]]
[[  0.         119.98533569]
 [120.19430845   0.        ]]
[[  0.         119.98533569]
 [120.19430845   0.        ]]
Means: min(0, envy)
[[ 0.         -3.57917165]
 [-3.90301165  0.        ]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
Stds
[[ 0.         88.75492692]
 [87.87904888  0.        ]]
[[ 0.         74.70661871]
 [73.26920199  0.        ]]
[[ 0.         74.70661871]
 [73.26920199  0.        ]]


In [90]:
(en0, en1, en2) = ef_percent(dir_uv_r6_u2)

Us shape : (1000, 2, 6)
Vs shape : (1000, 2, 3)
As shape : (1000, 2, 6)
EF0 percent :  48.699999999999996
EF1 percent :  98.0
EF2 percent :  100.0
Means: envy
[[ 0.         83.02162873]
 [74.47077309  0.        ]]
[[  0.         113.55231673]
 [107.61388393   0.        ]]
[[  0.        114.2853773]
 [109.0396012   0.       ]]
Means: max(0, envy)
[[ 0.         96.76593419]
 [90.96291689  0.        ]]
[[  0.         113.64229644]
 [107.8225121    0.        ]]
[[  0.        114.2853773]
 [109.0396012   0.       ]]
Means: min(0, envy)
[[  0.         -13.74430546]
 [-16.4921438    0.        ]]
[[ 0.         -0.08997971]
 [-0.20862817  0.        ]]
[[0. 0.]
 [0. 0.]]
Stds
[[  0.         107.64191776]
 [109.58075385   0.        ]]
[[ 0.         76.93305499]
 [76.92012276  0.        ]]
[[ 0.         76.22673622]
 [75.6337297   0.        ]]


In [ ]:
(en0, en1, en2) = ef_percent(dir_uv_r6_u4)

Us shape : (1000, 4, 6)
Vs shape : (1000, 4, 3)
As shape : (1000, 4, 6)
EF0 percent :  0.3
EF1 percent :  95.7


In [77]:
(en0, en1, en2) = ef_percent(dir_uv_r4_u2)

Us shape : (1000, 2, 4)
Vs shape : (1000, 2, 2)
As shape : (1000, 2, 4)
EF0 percent :  51.5
EF1 percent :  99.9
EF2 percent :  100.0
Means
[[ 0.         44.52326596]
 [41.32370711  0.        ]]
[[ 0.         75.12509966]
 [74.61377435  0.        ]]
[[ 0.         75.12509966]
 [74.67106896  0.        ]]
Stds
[[ 0.         62.65301931]
 [59.70746581  0.        ]]
[[ 0.         46.02593862]
 [44.43850919  0.        ]]
[[ 0.         46.02593862]
 [44.3773552   0.        ]]
